# AI Economy Score Predictor - Full Pipeline

Complete end-to-end implementation of the earnings call sentiment → economic prediction → trading strategy pipeline.

## Setup & Configuration

In [ ]:
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from data_acquisition import DataAcquisition
from llm_scorer import LLMScorer
from feature_engineering import FeatureEngineer
from prediction_model import PredictionModel
from signal_generator import SignalGenerator
from backtester import Backtester
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✓ Pipeline modules loaded")
print(f"✓ Config loaded: {len(config)} sections")

## Step 1: Data Acquisition

In [ ]:
# Initialize data acquisition
data_acq = DataAcquisition('config.yaml')
sp500 = data_acq.fetch_sp500_constituents()
sp500.head(10)

# Data Fetch Testing

In [ ]:
import pandas as pd
from data_acquisition import DataAcquisition
data = DataAcquisition("config.yaml")
transcripts = data.fetch_earnings_transcripts('2015-01-01', '2026-01-01')
print(f"Loaded {len(transcripts)} transcripts for Q1 2015")
macro = data.fetch_macro_data('2015-01-01', '2025-12-31')
print(f"Loaded {len(macro)} macro indicators")
sp500 = data.fetch_sp500_constituents()
print(f"Loaded {len(sp500)} S&P 500 stocks")

## Step 2: Fetch Macro Data (FRED API)

**Note**: If you get FRED API errors, restart the kernel to reload the config with the updated API key.

In [ ]:
# Fetch macroeconomic data
start_date = config['data']['transcripts']['start_date']
end_date = config['data']['transcripts']['end_date']
macro_data = data_acq.fetch_macro_data(start_date, end_date)
print(f"\n Macroeconomic Data:")
for name, df in macro_data.items():
    print(f"  {name}: {len(df)} observations")

In [ ]:
import pandas as pd
import re

pmi_path = 'pmi_data.csv'
pmi_df = pd.read_csv(pmi_path)
pmi_df.columns = [c.strip().lower().replace(' ', '_') for c in pmi_df.columns]
print("Columns in PMI file:", pmi_df.columns.tolist())
date_col = [col for col in pmi_df.columns if 'date' in col][0]
pmi_col = [col for col in pmi_df.columns if 'pmi' in col][0]
def clean_date(val):
    # Extract the part before the first parenthesis
    val = str(val).split('(')[0].strip()
    try:
        return pd.to_datetime(val)
    except Exception:
        return pd.NaT
pmi_df[date_col] = pmi_df[date_col].apply(clean_date)
pmi_df = pmi_df.dropna(subset=[date_col, pmi_col])
print(f"Loaded PMI data: {len(pmi_df)} rows")
print(pmi_df.tail())


In [ ]:
# Fetch control variables
controls = data_acq.fetch_control_variables(start_date, end_date)
print(f"\nControl Variables: {len(controls)} observations")
controls.head()

In [ ]:
data_acq.pmi_df = pmi_df
controls = data_acq.fetch_control_variables(start_date, end_date, pmi_df=pmi_df)

In [ ]:
controls.head()

In [ ]:
transcripts.head(1)

In [ ]:
transcripts['date'] = pd.to_datetime(transcripts['date'])
transcripts_2024_2025 = transcripts[
    (transcripts['date'] >= '2024-01-01') & 
    (transcripts['date'] <= '2025-12-31')
].copy()

print(f"Filtered to {len(transcripts_2024_2025)} transcripts (2024-2025)")
print(f"Date range: {transcripts_2024_2025['date'].min()} to {transcripts_2024_2025['date'].max()}")
print(f"\nBreakdown by year:")
print(transcripts_2024_2025['year'].value_counts().sort_index())


In [ ]:
transcripts_2024_2025.tail(10)

In [ ]:
for k in macro_data:
    print(k)

In [ ]:
macro_data['gdp'].tail(10)

In [ ]:
# count NAN 
macro_data['wages'].isna().sum()

## Step 2: LLM Scoring

In [ ]:
# Initialize LLM scorer
scorer = LLMScorer('config.yaml')

# Test text cleaning
sample_transcript = {
    'full_text': '''Forward-looking statements: This call contains forward-looking statements.
    
CEO: I'm pleased to report strong financial performance this quarter.
The US economy continues to show resilience despite some headwinds.
We see positive momentum in consumer spending and business investment.

Question-and-answer session:
Q: What's your outlook on the economy?
A: We remain cautiously optimistic about near-term growth.''',
    'md&a': 'Management discussion section...',
    'qa': 'Q&A section...'
}

# Clean transcript
cleaned = scorer.clean_transcript(sample_transcript['full_text'])
print("Cleaned transcript:")
print(cleaned[:200] + "...")

In [ ]:
# Extract MD&A section
md_a = scorer.extract_md_and_a(sample_transcript['full_text'])
print(f"MD&A section length: {len(md_a)} chars")
print(md_a[:150] + "...")

In [ ]:
# Chunk text for LLM processing
chunks = scorer.chunk_text(cleaned, chunk_size=500)
print(f"\nText chunked into {len(chunks)} pieces")
for i, chunk in enumerate(chunks[:2]):
    print(f"\nChunk {i+1} ({len(chunk)} chars):")
    print(chunk[:100] + "...")

## Step 3: Feature Engineering

In [ ]:
def aggregate_scores_by_quarter(scored_transcripts):
    """
    Aggregate individual transcript scores into quarterly AGG scores.
    
    Args:
        scored_transcripts: List of dicts with 'symbol', 'date', 'score', 'market_cap'
        
    Returns:
        DataFrame with quarterly AGG scores
    """
    df = pd.DataFrame(scored_transcripts)
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['quarter'] = df['date'].dt.quarter
    df['quarter_date'] = df['date'].dt.to_period('Q').dt.to_timestamp()
    
    # Aggregate by quarter using value-weighted average
    quarterly = df.groupby('quarter_date').apply(
        lambda x: np.average(x['score'], weights=x.get('market_cap', [1]*len(x)))
    ).reset_index()
    
    quarterly.columns = ['date', 'agg_score']
    quarterly['year'] = quarterly['date'].dt.year
    quarterly['quarter'] = quarterly['date'].dt.quarter
    
    return quarterly[['date', 'year', 'quarter', 'agg_score']]

# Example usage (commented out - requires real transcript scores):
# scored_transcripts = scorer.score_multiple_transcripts(transcripts)
# agg_scores = aggregate_scores_by_quarter(scored_transcripts)
# agg_scores.to_csv('agg_scores.csv', index=False)
print("✓ AGG score aggregation function defined")

## LLM Transcript Scoring - Choose Your Option

**OPTION A: Test Pipeline (2024-2025 only) - RECOMMENDED FIRST**
- Score approximately 2,500 transcripts (2 years)
- Cost: approximately $2.50-5.00 (GPT-4o-mini)
- Time: approximately 20-40 minutes
- Purpose: Test full pipeline before committing to full dataset

**OPTION B: Full Dataset (2015-2025)**
- Score approximately 13,600 transcripts (10 years)
- Cost: approximately $13-27 (GPT-4o-mini)
- Time: approximately 2-5 hours
- Purpose: Complete research dataset for publication-quality results

In [ ]:
# Choose scoring mode
TEST_MODE = True  # Set to False to run full dataset (2015-2025)

if TEST_MODE:
    # OPTION A: Test with 2024-2025 data
    print("TEST MODE: Checking for existing transcript data...")
    
    # Check if we already have filtered 2024-2025 data
    if 'transcripts_2024_2025' in dir() and len(transcripts_2024_2025) > 0:
        test_transcripts = transcripts_2024_2025.copy()
        print(f"Using pre-filtered transcripts_2024_2025 data: {len(test_transcripts)} transcripts")
    elif 'transcripts' in dir() and len(transcripts) > 0:
        # Filter existing transcripts to 2024-2025
        print("Filtering full transcript data to 2024-2025...")
        transcripts_copy = transcripts.copy()
        transcripts_copy['date'] = pd.to_datetime(transcripts_copy['date'])
        test_transcripts = transcripts_copy[
            (transcripts_copy['date'] >= '2024-01-01') & 
            (transcripts_copy['date'] <= '2025-12-31')
        ].copy()
        print(f"Filtered {len(transcripts)} → {len(test_transcripts)} transcripts")
    else:
        print("No transcripts loaded yet, fetching 2024-2025...")
        test_transcripts = data_acq.fetch_earnings_transcripts('2024-01-01', '2025-12-31')
    
    print(f"\nTotal transcripts to score: {len(test_transcripts)}")
    print(f"  Estimated cost: ${len(test_transcripts) * 0.001:.2f} - ${len(test_transcripts) * 0.002:.2f}")
    print(f"  Estimated time: {len(test_transcripts) * 2 / 60:.1f} - {len(test_transcripts) * 3 / 60:.1f} minutes")
    print(f"\nData will be saved to: test_scored_transcripts_2024_2025.csv")
    
    # Show breakdown by year
    test_transcripts['year'] = pd.to_datetime(test_transcripts['date']).dt.year
    year_counts = test_transcripts['year'].value_counts().sort_index()
    print(f"\nTranscripts by year:")
    for year, count in year_counts.items():
        print(f"  {year}: {count} transcripts")
    
    scoring_transcripts = test_transcripts
    save_path = 'test_scored_transcripts_2024_2025.csv'
    
else:
    # OPTION B: Full dataset (2015-2025)
    print("FULL MODE: Checking for existing transcript data...")
    
    # Check if we already have full dataset loaded
    if 'transcripts' in dir() and len(transcripts) > 0:
        transcripts_copy = transcripts.copy()
        transcripts_copy['date'] = pd.to_datetime(transcripts_copy['date'])
        date_range = (transcripts_copy['date'].min(), transcripts_copy['date'].max())
        
        # Check if we have enough coverage
        if date_range[0] <= pd.Timestamp('2015-01-01') and date_range[1] >= pd.Timestamp('2025-01-01'):
            print(f"Reusing {len(transcripts_copy)} transcripts from already-loaded data")
            print(f"  Date range: {date_range[0].date()} to {date_range[1].date()}")
            all_transcripts = transcripts_copy[
                (transcripts_copy['date'] >= '2015-01-01') & 
                (transcripts_copy['date'] <= '2025-12-31')
            ]
        else:
            print(f"Loaded data has limited range ({date_range[0].date()} to {date_range[1].date()})")
            print("Fetching complete 2015-2025 dataset...")
            all_transcripts = data_acq.fetch_earnings_transcripts('2015-01-01', '2025-12-31')
    else:
        print("No transcripts loaded yet, fetching 2015-2025...")
        all_transcripts = data_acq.fetch_earnings_transcripts('2015-01-01', '2025-12-31')
    
    print(f"\nTotal transcripts to score: {len(all_transcripts)}")
    print(f"  Estimated cost: ${len(all_transcripts) * 0.001:.2f} - ${len(all_transcripts) * 0.002:.2f}")
    print(f"  Estimated time: {len(all_transcripts) * 2 / 3600:.1f} - {len(all_transcripts) * 3 / 3600:.1f} hours")
    print(f"\nData will be saved to: all_scored_transcripts_2015_2025.csv")
    
    # Show breakdown by year
    all_transcripts['year'] = pd.to_datetime(all_transcripts['date']).dt.year
    year_counts = all_transcripts['year'].value_counts().sort_index()
    print(f"\nTranscripts by year:")
    for year, count in year_counts.items():
        print(f"  {year}: {count} transcripts")
    
    scoring_transcripts = all_transcripts
    save_path = 'all_scored_transcripts_2015_2025.csv'

print(f"\nReady to score {len(scoring_transcripts)} transcripts")
print(f"Checkpoints will be saved every 50 transcripts")

In [ ]:
# Define the scoring function with progress tracking
import time
from tqdm.notebook import tqdm
from datetime import datetime

def score_quarter_transcripts(transcripts_df, scorer, save_path='scored_transcripts.csv'):
    """
    Score all transcripts with progress tracking, checkpointing, and error handling.
    """
    # First, inspect the data structure
    print("Inspecting data structure...")
    print(f"Type: {type(transcripts_df)}")
    print(f"Columns: {transcripts_df.columns.tolist()}")
    print(f"\nFirst row type: {type(transcripts_df.iloc[0])}")
    print(f"First row preview:")
    print(transcripts_df.iloc[0])
    
    print(f"\nScoring {len(transcripts_df)} transcripts...")
    print(f"Estimated cost: ${len(transcripts_df) * 0.001:.2f} (GPT-4o-mini)")
    print(f"Estimated time: {len(transcripts_df) * 2 / 60:.1f} minutes")
    
    # Check for existing progress
    try:
        existing = pd.read_csv(save_path)
        already_scored = set(existing['symbol'] + '_' + existing['date'].astype(str))
        print(f"Found {len(already_scored)} previously scored transcripts")
    except FileNotFoundError:
        already_scored = set()
        existing = pd.DataFrame()
    
    scored_results = []
    errors = []
    
    # Determine transcript column name - check what's actually in the DataFrame
    available_cols = transcripts_df.columns.tolist()
    transcript_col = None
    
    for possible_name in ['transcript', 'text', 'content', 'full_text', 'body']:
        if possible_name in available_cols:
            transcript_col = possible_name
            break
    
    if transcript_col is None:
        print(f"ERROR: Could not find transcript column. Available columns: {available_cols}")
        return existing if len(existing) > 0 else pd.DataFrame()
    
    print(f"Using transcript column: '{transcript_col}'")
    
    # Convert to dict records for easier iteration
    records = transcripts_df.to_dict('records')
    
    for idx, row in enumerate(tqdm(records, desc="Scoring")):
        # Handle different possible column names
        symbol = row.get('symbol') or row.get('ticker') or 'UNKNOWN'
        date = row.get('date') or row.get('filing_date') or 'UNKNOWN'
        transcript_id = f"{symbol}_{date}"
        
        # Skip if already scored
        if transcript_id in already_scored:
            continue
        
        try:
            # Get the transcript text
            transcript_text = row.get(transcript_col, '')
            
            if not transcript_text or transcript_text == '':
                errors.append({'symbol': symbol, 'date': date, 'error': 'Empty transcript'})
                continue
            
            # Score transcript - wrap in expected dictionary format
            # The scorer expects a dict with 'full_text' key
            transcript_dict = {'full_text': transcript_text}
            result = scorer.score_transcript(transcript_dict, use_md_a_only=False)
            score = result['firm_score']
            
            if score is None:
                errors.append({'symbol': symbol, 'date': date, 'error': 'Scoring returned None'})
                continue
            
            scored_results.append({
                'symbol': symbol,
                'date': date,
                'score': score,
                'transcript_length': len(str(transcript_text))
            })
            
            # Save checkpoint every 50 transcripts
            if len(scored_results) % 50 == 0:
                temp_df = pd.DataFrame(scored_results)
                combined = pd.concat([existing, temp_df], ignore_index=True)
                combined.to_csv(save_path, index=False)
                print(f"\nCheckpoint: Saved {len(combined)} scores")
            
            # Rate limiting (to avoid API limits)
            time.sleep(0.5)
            
        except Exception as e:
            errors.append({'symbol': symbol, 'date': date, 'error': str(e)})
            if idx < 5:  # Only print first few errors in detail
                print(f"\nError scoring {symbol}: {e}")
    
    # Final save - handle case where nothing was scored
    if scored_results:
        final_df = pd.DataFrame(scored_results)
        combined = pd.concat([existing, final_df], ignore_index=True)
        combined.to_csv(save_path, index=False)
        print(f"\nSaved {len(combined)} total scored transcripts to {save_path}")
    elif len(existing) > 0:
        combined = existing
        print(f"\nNo new transcripts scored. Returning {len(existing)} existing scores.")
    else:
        combined = pd.DataFrame(columns=['symbol', 'date', 'score', 'transcript_length'])
        print("\nWARNING: No transcripts were scored successfully!")
    
    if errors:
        error_df = pd.DataFrame(errors)
        error_df.to_csv('scoring_errors.csv', index=False)
        print(f"\nWARNING: {len(errors)} errors occurred (saved to scoring_errors.csv)")
        print(f"First few unique errors:")
        unique_errors = error_df['error'].value_counts().head(3)
        for error_msg, count in unique_errors.items():
            print(f"  {error_msg}: {count} occurrences")
    
    return combined

print("Scoring function ready")

In [ ]:
# Inspect the data structure before scoring (Optional)
print("Data structure inspection:")
print(f"Type of scoring_transcripts: {type(scoring_transcripts)}")
print(f"Shape: {scoring_transcripts.shape}")
print(f"Columns: {scoring_transcripts.columns.tolist()}")
print(f"\nFirst transcript preview:")
print(scoring_transcripts.iloc[0])

In [ ]:
print(f"Starting scoring at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*70)

scored_data = score_quarter_transcripts(
    scoring_transcripts, 
    scorer, 
    save_path=save_path
)

print("="*70)
print(f"Completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nFinal Results:")
print(f"  Total scored: {len(scored_data)}")
print(f"  Date range: {scored_data['date'].min()} to {scored_data['date'].max()}")
print(f"  Average score: {scored_data['score'].mean():.2f}")
print(f"  Score distribution:")
print(scored_data['score'].value_counts().sort_index())
print(f"\nSaved to: {save_path}")

In [ ]:
# Aggregate scored transcripts into quarterly AGG scores
print("Aggregating individual scores into quarterly AGG scores...")

# Convert to DataFrame if needed
if isinstance(scored_data, pd.DataFrame):
    scored_df = scored_data.copy()
else:
    scored_df = pd.DataFrame(scored_data)

# Ensure date column is datetime
scored_df['date'] = pd.to_datetime(scored_df['date'])
scored_df['year'] = scored_df['date'].dt.year
scored_df['quarter'] = scored_df['date'].dt.quarter

# Group by quarter and calculate aggregate score
agg_scores = scored_df.groupby(['year', 'quarter']).agg({
    'score': ['mean', 'std', 'count']
}).reset_index()

agg_scores.columns = ['year', 'quarter', 'agg_score', 'score_std', 'num_firms']

# Create quarter date
agg_scores['date'] = pd.to_datetime(
    agg_scores['year'].astype(str) + '-Q' + agg_scores['quarter'].astype(str)
)

# Reorder columns
final_agg_scores = agg_scores[['date', 'year', 'quarter', 'agg_score', 'score_std', 'num_firms']]

# Save AGG scores
agg_filename = 'test_agg_scores_2024_2025.csv' if TEST_MODE else 'agg_scores_2015_2025.csv'
final_agg_scores.to_csv(agg_filename, index=False)
print(f"\nSUCCESS: Saved {len(final_agg_scores)} quarterly AGG scores to {agg_filename}")

# Display results
print(f"\nAGG Scores Summary:")
print(final_agg_scores)
print(f"\nStatistics:")
print(f"  Quarters covered: {len(final_agg_scores)}")
print(f"  Date range: {final_agg_scores['date'].min().strftime('%Y-%m-%d')} to {final_agg_scores['date'].max().strftime('%Y-%m-%d')}")
print(f"  Mean AGG score: {final_agg_scores['agg_score'].mean():.3f}")
print(f"  Std AGG score: {final_agg_scores['agg_score'].std():.3f}")
print(f"  Average firms/quarter: {final_agg_scores['num_firms'].mean():.0f}")

In [ ]:
# Initialize feature engineer
engineer = FeatureEngineer('config.yaml')

# Load real AGG scores from saved file or create from actual transcript scoring
try:
    agg_scores = pd.read_csv('agg_scores.csv')
    agg_scores['date'] = pd.to_datetime(agg_scores['date'])
    print(f"✓ Loaded real AGG scores from file: {len(agg_scores)} quarters")
    print(agg_scores.head())
except FileNotFoundError:
    print("⚠ No saved AGG scores found. You need to:")
    print("  1. Score earnings transcripts using LLMScorer.score_multiple_transcripts()")
    print("  2. Aggregate scores by quarter using aggregate_scores_by_quarter()")
    print("  3. Save to 'agg_scores.csv'")
    print("\n For demonstration, showing expected data structure...")
    # Show expected structure instead of generating synthetic data
    agg_scores = pd.DataFrame({
        'date': pd.date_range(start='2015-01-01', end='2023-12-31', freq='Q'),
        'year': [],
        'quarter': [],
        'agg_score': []  # Real scores would be 1-5 from LLM
    })
    print("\nExpected columns: date, year, quarter, agg_score")
    print("Cannot proceed with feature engineering without real data")

In [ ]:
# Normalize scores (only if we have real data)
if len(agg_scores) > 0 and 'agg_score' in agg_scores.columns:
    normalized = engineer.normalize_scores(agg_scores, method='zscore', window=20)
    print("\nNormalized Scores:")

    print(normalized[['date', 'agg_score', 'agg_score_norm']].head(10))    normalized = pd.DataFrame()

else:    print("⚠ Cannot normalize without real AGG scores")

In [ ]:
# Create delta features (only if we have normalized data)
if len(normalized) > 0:
    with_deltas = engineer.create_delta_features(normalized)
    print("\nDelta Features:")

    print(with_deltas[['date', 'agg_score', 'yoy_change', 'qoq_change', 'momentum']].tail(10))    with_deltas = pd.DataFrame()

else:    print("⚠ Cannot create delta features without normalized scores")

In [ ]:
# Visualize AGG score and deltas (only if we have features)
if len(with_deltas) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(12, 8))

    # AGG score
    axes[0].plot(with_deltas['date'], with_deltas['agg_score'], linewidth=2)
    axes[0].set_title('AGG Score (National Economic Sentiment)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Score')
    axes[0].grid(True, alpha=0.3)

    # YoY change
    valid_yoy = with_deltas.dropna(subset=['yoy_change'])
    axes[1].bar(valid_yoy['date'], valid_yoy['yoy_change'], color='steelblue', alpha=0.7)
    axes[1].set_title('YoY Change (AGG_t - AGG_t-4)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Change')
    axes[1].grid(True, alpha=0.3)

    # Momentum
    valid_momentum = with_deltas.dropna(subset=['momentum'])
    axes[2].bar(valid_momentum['date'], valid_momentum['momentum'], color='coral', alpha=0.7)
    axes[2].set_title('Momentum (Acceleration)', fontsize=12, fontweight='bold')
    axes[2].set_ylabel('Momentum')
    axes[2].set_xlabel('Date')
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("✓ Feature visualization complete")
else:
    print("⚠ Cannot visualize features without delta features")

## Step 4: Prediction Models

In [ ]:
pred_model = PredictionModel('config.yaml')
print(dir(pred_model))

In [ ]:
X_train = with_deltas[['agg_score_norm', 'yoy_change', 'qoq_change', 'momentum']].dropna().reset_index(drop=True)
X_train['date'] = with_deltas.loc[X_train.index, 'date'].values

gdp_df = macro_data['gdp'].copy()
gdp_df['date'] = pd.to_datetime(gdp_df['date'])
train_data = X_train.merge(gdp_df, on='date', how='inner')
X_train = train_data[['agg_score_norm', 'yoy_change', 'qoq_change', 'momentum']].values
y_train = train_data['value'].values
print(f"Training data: {X_train.shape}")
print(f"Target data: {y_train.shape}")
gdp_models = pred_model.train_gdp_models(X_train, y_train)
print(f"Model R²: {gdp_models['gdp'].score(X_train, y_train):.3f}")
gdp_model = pred_model.train_gdp_model(X_train.values, y_train.values)
print(f"Training data: {X_train.shape}")
print(f"Target data: {y_train.shape}")

In [ ]:
# Train GDP prediction model
gdp_model = pred_model.train_gdp_model(X_train, y_train)
print(f"\nGDP Model Trained")
print(f"  Model type: {type(gdp_model).__name__}")
print(f"  Training R²: {gdp_model.score(X_train, y_train):.3f}")

In [ ]:
# Make predictions using real test data
if len(agg_scores) > 0 and 'agg_score' in agg_scores.columns:
    # Use the most recent features for out-of-sample prediction
    test_features = with_deltas[['agg_score_norm', 'yoy_change', 'qoq_change', 'momentum']].dropna().tail(10)
    test_dates = with_deltas.loc[test_features.index, 'date']
    
    predictions = gdp_model.predict(test_features.values)

    print(f"\nGDP Predictions (1Q ahead) for recent quarters:")
    for date, pred in zip(test_dates, predictions):
        print(f"  {date.strftime('%Y-%m-%d')}: {pred:.3f}%")
    print(f"\n  Mean: {predictions.mean():.3f}%")
    print(f"  Std: {predictions.std():.3f}%")
    print(f"  Range: [{predictions.min():.3f}, {predictions.max():.3f}]%")
else:
    print("⚠ Cannot make predictions without real AGG scores")

## Step 5: Signal Generation & Backtesting

In [ ]:
# Initialize signal generator
signal_gen = SignalGenerator('config.yaml')

# Use real predictions from trained models
# This requires: 
# 1. Features from AGG scores
# 2. Trained GDP/IP models
# 3. SPF forecasts from data_acq.fetch_spf_forecasts()

if len(agg_scores) > 0 and 'agg_score' in agg_scores.columns:
    # Use real model predictions
    features_for_pred = with_deltas[['agg_score_norm', 'yoy_change', 'qoq_change', 'momentum']].dropna()
    dates_for_pred = with_deltas.loc[features_for_pred.index, 'date']
    

    # Get predictions from trained model    predictions_df = pd.DataFrame()

    gdp_predictions = gdp_model.predict(features_for_pred.values)    print("⚠ Cannot generate predictions without real AGG scores")

    else:

    # Fetch real SPF forecasts    print(predictions_df.head())

    try:    print("✓ Real Predictions vs SPF:")

        spf_data = data_acq.fetch_spf_forecasts(start_date, end_date)    

        spf_data['date'] = pd.to_datetime(spf_data['date'])    predictions_df.rename(columns={'rgdp_1q': 'gdp_spf'}, inplace=True)

    except Exception as e:    predictions_df = predictions_df.merge(spf_data[['date', 'rgdp_1q']], on='date', how='left')

        print(f"⚠ Could not fetch SPF data: {e}")    })

        spf_data = pd.DataFrame({'date': dates_for_pred, 'rgdp_1q': [2.0]*len(dates_for_pred)})        'gdp_pred': gdp_predictions

            'date': dates_for_pred.values,

    # Combine predictions with SPF    predictions_df = pd.DataFrame({

In [ ]:
# Generate trading signals (only if we have real predictions)
if len(predictions_df) > 0:
    signals = signal_gen.generate_signals(predictions_df)
    print(f"\n📊 Trading Signals Generated:")
    print(signals.head(10))
    print(f"\nSignal distribution:")
    print(signals['signal'].value_counts())
else:
    print("⚠ Cannot generate signals without predictions")
    signals = pd.DataFrame()

In [ ]:
# Initialize backtester
backtester = Backtester('config.yaml')

# Use real returns from strategy execution
# This requires:
# 1. Trading signals from signal_gen.generate_signals()
# 2. Sector ETF price data
# 3. Portfolio construction and rebalancing

if len(predictions_df) > 0:
    # Fetch real ETF price data for sectors
    sector_etfs = config['strategy']['sector_etfs']
    etf_start = config['backtest']['test_start']
    etf_end = config['backtest']['test_end']
    
    etf_prices = data_acq.fetch_etf_prices(sector_etfs, etf_start, etf_end)
    
    if etf_prices:
        print(f"✓ Fetched price data for {len(etf_prices)} sector ETFs")

                    print(f"  {metric}: {value}")

        # Run backtest with real data        else:

        # Note: This requires implementing the full backtesting logic            print(f"  {metric}: {value:.3f}")

        # For now, we show the structure        if isinstance(value, float):

        print("\n⚠ Full backtest execution requires:")    for metric, value in metrics.items():

        print("  1. Signals from signal_gen.generate_signals(predictions_df)")    print(f"\n📈 Performance Metrics:")

        print("  2. Portfolio construction based on signals")    metrics = backtester.calculate_metrics(portfolio_returns)

        print("  3. Daily rebalancing and return calculation")    # Calculate performance metrics

        print("  4. Benchmark comparison (SPY or equal-weight)")if len(portfolio_returns) > 0:

        

        portfolio_returns = pd.DataFrame()    portfolio_returns = pd.DataFrame()

        print("\nPlease implement backtester.run_backtest(signals, etf_prices) for real returns")    print("⚠ Cannot run backtest without predictions")

    else:else:

        print("⚠ No ETF price data available")        portfolio_returns = pd.DataFrame()

In [ ]:
# Calculate cumulative returns and plot (only if we have real returns)
if len(portfolio_returns) > 0 and 'strategy_return' in portfolio_returns.columns:
    portfolio_returns['strategy_cumret'] = (1 + portfolio_returns['strategy_return']).cumprod() - 1
    portfolio_returns['benchmark_cumret'] = (1 + portfolio_returns['benchmark_return']).cumprod() - 1

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(portfolio_returns['date'], portfolio_returns['strategy_cumret'] * 100, 
            label='Strategy', linewidth=2)
    ax.plot(portfolio_returns['date'], portfolio_returns['benchmark_cumret'] * 100, 
            label='Benchmark', linewidth=2, linestyle='--')

    ax.set_title('Strategy vs Benchmark Cumulative Returns', fontsize=12, fontweight='bold')
    ax.set_ylabel('Return (%)')
    ax.set_xlabel('Date')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


    print("✓ Backtest visualization complete")    print("5. Execute backtest with real ETF prices")

else:    print("4. Generate trading signals")

    print("⚠ No portfolio returns available for visualization")    print("3. Train prediction models")

    print("\nTo complete the full pipeline with real data:")    print("2. Engineer features from AGG scores")
    print("1. Score earnings transcripts → agg_scores.csv")

## Summary: Complete Pipeline with Real Data

This notebook demonstrates the **AI Economy Score Predictor** strategy pipeline using **real data sources**:

### ✅ Real Data Used:
1. **Macroeconomic Data**: From FRED API (GDP, Industrial Production, Employment, Wages)
2. **Control Variables**: From FRED API (Yield Curve, Consumer Sentiment, Unemployment)
3. **PMI Data**: Loaded from `pmi_data.csv` 
4. **S&P 500 Constituents**: From `constituents.csv`
5. **ETF Prices**: Fetched via yfinance API

### ⚠️ Real Data Needed:
- **Earnings Call Transcripts** with LLM sentiment scores aggregated quarterly → `agg_scores.csv`

### Pipeline Steps:
1. **Data Acquisition** ✓ Uses real FRED API and local files
2. **LLM Scoring** → Requires real earnings transcripts (Seeking Alpha, CapIQ, Bloomberg)
3. **Feature Engineering** ✓ Works with real AGG scores once available
4. **Prediction Models** ✓ Trains on real macro data + AGG features
5. **Signal Generation** ✓ Compares predictions to SPF forecasts
6. **Backtesting** ✓ Uses real sector ETF prices

### Next Steps:
1. Obtain earnings call transcripts from a data provider
2. Score transcripts using `LLMScorer.score_multiple_transcripts()`
3. Aggregate scores by quarter and save to `agg_scores.csv`
4. Re-run this notebook to execute the full pipeline with real signals

**No synthetic/random data is used for actual trading signals - all results require real transcript scoring.**

In [ ]:
# Check data availability
import os

print("📁 Data File Status:\n")

required_files = {
    'config.yaml': 'Configuration file',
    'constituents.csv': 'S&P 500 constituents',
    'pmi_data.csv': 'PMI data'
}

optional_files = {
    'agg_scores.csv': 'Aggregated LLM sentiment scores (REQUIRED for full pipeline)'
}

for file, desc in required_files.items():
    status = "✓" if os.path.exists(file) else "✗"
    print(f"{status} {file}: {desc}")

print("\nOptional (but critical):")
for file, desc in optional_files.items():
    status = "✓" if os.path.exists(file) else "✗ MISSING"
    print(f"{status} {file}: {desc}")

if not os.path.exists('agg_scores.csv'):
    print("\n⚠️  To create agg_scores.csv, you need to:")
    print("   1. Get earnings transcripts from a data provider")
    print("   2. Run LLM scoring (see 'Note: To Use Real Data' section above)")
    print("   3. Use the aggregate_scores_by_quarter() function")